# COP Analysis FMA-P

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
import re
from scipy.signal import butter, filtfilt

    The exported COP data from the PKMas software has already been filtered. Filter applied to raw COP data: 2 pass 5th order Low Pass Butterworth Filter with zero lag at 10 Hz

In [ ]:
# Control data
mmmp_dir = './Control'
fma_control_cop = glob(os.path.join(mmmp_dir, '**/FMA0[0-9]_C[0-9]*.txt'), recursive=True) #COP files

#MoCap data
fma_control_su = glob(os.path.join(mmmp_dir, '**/StandingUp_Chair_FMA0[0-9]_C[0-9]*.tsv'), recursive=True)
fma_control_pk = glob(os.path.join(mmmp_dir, '**/Picking_Key_FMA0[0-9]_C[0-9]*.tsv'), recursive=True)
fma_control_t = glob(os.path.join(mmmp_dir, '**/Turning_FMA0[0-9]_C[0-9]*.tsv'), recursive=True)
fma_control_standtosit = glob(os.path.join(mmmp_dir, '**/SittingDown_Chair_FMA0[0-9]_C[0-9]*.tsv'), recursive=True)




In [ ]:
#PwP data
mmmp_dir = './PwP'
fma_pwp_cop = glob(os.path.join(mmmp_dir, '**/FMA0[0-9]_P[0-9]*.txt'), recursive=True) #COP files

#MoCap data
fma_pwp_su = glob(os.path.join(mmmp_dir, '**/StandingUp_Chair_FMA0[0-9]_P[0-9]*.tsv'), recursive=True)
fma_pwp_pk = glob(os.path.join(mmmp_dir, '**/Picking_Key_FMA0[0-9]_P[0-9]*.tsv'), recursive=True)
fma_pwp_t = glob(os.path.join(mmmp_dir, '**/Turning_FMA0[0-9]_P[0-9]*.tsv'), recursive=True)
fma_pwp_standtosit = glob(os.path.join(mmmp_dir, '**/SittingDown_Chair_FMA0[0-9]_P[0-9]*.tsv'), recursive=True)


In [ ]:
def read_cop_file(file_path):
    df = pd.read_csv(file_path, skiprows=10, sep=';')
    # Convert 'Time (sec.)' column to numeric, coercing non-numeric values to NaN
    df['Time (sec.)'] = pd.to_numeric(df['Time (sec.)'], errors='coerce')
    
    # Extract the 'Time (sec.)' values as a NumPy array
    time = df['Time (sec.)'].to_numpy()
    
    cop = np.zeros((len(time), 2))
    # Convert the cop columns to numeric, coercing non-numeric values to NaN
    df['COP Both X'] = pd.to_numeric(df['COP Both X'], errors='coerce')
    df['COP Both Y'] = pd.to_numeric(df['COP Both Y'], errors='coerce')
    
    cop = df.loc[:, ['COP Both X', 'COP Both Y']].to_numpy()
    
    return time,cop

In [ ]:
def time_flag_mocap_file(mocap_path):
        # Load the motion capture data into a Pandas DataFrame
    df = pd.read_csv(mocap_path,sep = '\t',skiprows=11)
    time =  df.loc[:, "Time"].to_numpy()
    init_time = time[0]
    final_time = time[-1]
    return init_time, final_time

In [ ]:
def find_matching_file_path(cop_file_path, list_file_paths):
    # Extract the common part of the cop filename
    common_filename = os.path.basename(cop_file_path).split('.')[0]
#     print(common_filename)
    #     # Search for the corresponding turning file with an exact match
    matching_turning_files = [file for file in list_file_paths if common_filename in file]
    
    # Return the first matching turning file if found, otherwise return None
    return matching_turning_files[0] if matching_turning_files else None

path = find_matching_file_path(fma_pwp_cop[-2],fma_pwp_pk)
print(fma_pwp_cop[-2])
print(path)
# FMA03_P8 has no picking up key file

## Main functions

In [ ]:
def crop_cop_array(time, cop, init_turn, fin_turn, tolerance=1e-4):
    # Check if time array contains numeric values
    flag = True
    if not np.issubdtype(time.dtype, np.number):
        print("Error: Time array does not contain numeric values.")
        flag = False
        return flag, None, None

    # Find indices corresponding to init_turn and fin_turn in the time variable
    init_indices = np.where(np.isclose(time, init_turn, atol=tolerance))[0]
    fin_indices = np.where(np.isclose(time, fin_turn, atol=tolerance))[0]

    # Check if any indices are found
    if init_indices.size == 0 or fin_indices.size == 0:
        print("Error: Initial or final time not found in the time array.")
        flag = False
        return flag, None, None

    # Use the first index found
    init_index = init_indices[0]
    fin_index = fin_indices[0]

    # Crop COP array using init_index and fin_index
    cropped_cop = cop[init_index:fin_index + 1, :]
    cropped_time = time[init_index:fin_index + 1]

    return flag, cropped_cop, cropped_time


def get_AP_ML_cop(path,cop_file):
    
    if path is not None:
        # Proceed with the rest of  analysis, including time flags and cropping

        init_turn, fin_turn = time_flag_mocap_file(path)
#         print("Initial Time:", init_turn)
#         print("Final Time:", fin_turn)

        time, cop = read_cop_file(cop_file)

        # Crop COP array
        flag,cropped_cop,cropped_time = crop_cop_array(time, cop, init_turn, fin_turn)
#         print(type(cropped_cop))
        if flag:
            cropped_cop -= cropped_cop[0]
            peak_cop_ML = max(np.abs(cropped_cop[:,0]))
            mean_cop_ML = np.mean(cropped_cop[:,0])
            std_cop_ML = np.std(cropped_cop[:,0])
            
            peak_cop_AP = max(np.abs(cropped_cop[:,1]))
            mean_cop_AP = np.mean(cropped_cop[:,1])
            std_cop_AP = np.std(cropped_cop[:,1])


            return peak_cop_ML, mean_cop_ML, std_cop_ML, peak_cop_AP, mean_cop_AP, std_cop_AP
        else:
            return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
    else: 
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
    

def get_mean_displacement_velocity(path, cop_file):
    
    if path is not None:
        # Proceed with the rest of the analysis, including time flags and cropping

        init_turn, fin_turn = time_flag_mocap_file(path)
        time, cop = read_cop_file(cop_file)

        # Crop COP array
        flag,cropped_cop,cropped_time = crop_cop_array(time, cop, init_turn, fin_turn)

        if flag:
            cropped_cop -= cropped_cop[0]
            
            # Calculate mean displacement velocity
            mean_velocity_ML = np.abs(np.mean(np.diff(cropped_cop[:, 0]) / np.diff(cropped_time)))
            mean_velocity_AP = np.abs(np.mean(np.diff(cropped_cop[:, 1]) / np.diff(cropped_time)))

            return mean_velocity_ML, mean_velocity_AP,cropped_cop
        else:
            return 0, 0
    else: 
        return 0, 0


## COP displacement on Picking-up key task

In [ ]:
#Create the dataframes to store the shoulder tilt and stroop angles
df_cop_pick_key = pd.DataFrame(columns=['Subject ID','Group','TrialFMA',
                                   'Peak_COP_displacement_ML[mm]','Peak_COP_displacement_AP[mm]',
                                        'Mean_COP_displacement_ML[mm]','Mean_COP_displacement_AP[mm]',
                                        'std_COP_displacement_ML[mm]','std_COP_displacement_AP[mm]'
                                       ])

full_data = [fma_control_pk,fma_pwp_pk]
cop_data = [fma_control_cop,fma_pwp_cop]
data_label = ['Control','PwP']

for i,group in enumerate(cop_data):
    type_subject = data_label[i]
    for file in group:
                
       # Extract subject ID
        if type_subject == 'Control':
            subject_id = re.search(r'_C(\d+)\.txt$', file).group(1)
            subject_id = 'C'+subject_id
        else: 
            subject_id = re.search(r'_P(\d+)\.txt$', file).group(1)
            subject_id = 'P'+subject_id


        # Extract trial ID
        trial = re.search(r'FMA0(\d+)_', file).group(1)
#         print(subject_id,trial)
        cop_file = file
        path = find_matching_file_path(cop_file,full_data[i])
        peak_cop_ML, mean_cop_ML, std_cop_ML, peak_cop_AP, mean_cop_AP, std_cop_AP = get_AP_ML_cop(path,cop_file)

        new_line = {'Subject ID':subject_id,'Group':type_subject,'TrialFMA':trial,
                   'Peak_COP_displacement_ML[mm]':peak_cop_ML,
                    'Peak_COP_displacement_AP[mm]':peak_cop_AP,
                    'Mean_COP_displacement_ML[mm]':mean_cop_ML,
                    'Mean_COP_displacement_AP[mm]':mean_cop_AP,
                    'std_COP_displacement_ML[mm]':std_cop_ML,
                    'std_COP_displacement_AP[mm]':std_cop_AP
                   }
        
        df_cop_pick_key.loc[len(df_cop_pick_key)] = new_line

In [ ]:
df_cop_pick_key

## COP displacement on Turning task

In [ ]:
#Create the dataframes to store the shoulder tilt and stroop angles
df_cop_turning = pd.DataFrame(columns=['Subject ID','Group','TrialFMA',
                                   'Peak_COP_displacement_ML[mm]','Peak_COP_displacement_AP[mm]',
                                        'Mean_COP_displacement_ML[mm]','Mean_COP_displacement_AP[mm]',
                                        'std_COP_displacement_ML[mm]','std_COP_displacement_AP[mm]'
                                      ])

full_data = [fma_control_t,fma_pwp_t]
cop_data = [fma_control_cop,fma_pwp_cop]
data_label = ['Control','PwP']

for i,group in enumerate(cop_data):
    type_subject = data_label[i]
    for file in group:
                
       # Extract subject ID
        if type_subject == 'Control':
            subject_id = re.search(r'_C(\d+)\.txt$', file).group(1)
            subject_id = 'C'+subject_id
        else: 
            subject_id = re.search(r'_P(\d+)\.txt$', file).group(1)
            subject_id = 'P'+subject_id


        # Extract trial ID
        trial = re.search(r'FMA0(\d+)_', file).group(1)
#         print(subject_id,trial)
        cop_file = file
        path = find_matching_file_path(cop_file,full_data[i])
#         print(cop_file)
#         print(path)
        peak_cop_ML, mean_cop_ML, std_cop_ML, peak_cop_AP, mean_cop_AP, std_cop_AP = get_AP_ML_cop(path,cop_file)

        new_line = {'Subject ID':subject_id,'Group':type_subject,'TrialFMA':trial,
                   'Peak_COP_displacement_ML[mm]':peak_cop_ML,
                    'Peak_COP_displacement_AP[mm]':peak_cop_AP,
                    'Mean_COP_displacement_ML[mm]':mean_cop_ML,
                    'Mean_COP_displacement_AP[mm]':mean_cop_AP,
                    'std_COP_displacement_ML[mm]':std_cop_ML,
                    'std_COP_displacement_AP[mm]':std_cop_AP
                   }
        
        df_cop_turning.loc[len(df_cop_turning)] = new_line

In [ ]:
df_cop_turning

## COP displacement on Sit-to-stand task

In [ ]:
#Create the dataframes to store the shoulder tilt and stroop angles
df_cop_stand_up = pd.DataFrame(columns=['Subject ID','Group','TrialFMA',
                                   'Peak_COP_displacement_ML[mm]','Peak_COP_displacement_AP[mm]',
                                        'Mean_COP_displacement_ML[mm]','Mean_COP_displacement_AP[mm]',
                                        'std_COP_displacement_ML[mm]','std_COP_displacement_AP[mm]'
                                       ])

full_data = [fma_control_su,fma_pwp_su]
cop_data = [fma_control_cop,fma_pwp_cop]
data_label = ['Control','PwP']

for i,group in enumerate(cop_data):
    type_subject = data_label[i]
    for file in group:
                
       # Extract subject ID
        if type_subject == 'Control':
            subject_id = re.search(r'_C(\d+)\.txt$', file).group(1)
            subject_id = 'C'+subject_id
        else: 
            subject_id = re.search(r'_P(\d+)\.txt$', file).group(1)
            subject_id = 'P'+subject_id


        # Extract trial ID
        trial = re.search(r'FMA0(\d+)_', file).group(1)
#         print(subject_id,trial)
        cop_file = file
        path = find_matching_file_path(cop_file,full_data[i])
#         print(cop_file)
#         print(path)
        peak_cop_ML, mean_cop_ML, std_cop_ML, peak_cop_AP, mean_cop_AP, std_cop_AP = get_AP_ML_cop(path,cop_file)

        new_line = {'Subject ID':subject_id,'Group':type_subject,'TrialFMA':trial,
                   'Peak_COP_displacement_ML[mm]':peak_cop_ML,
                    'Peak_COP_displacement_AP[mm]':peak_cop_AP,
                    'Mean_COP_displacement_ML[mm]':mean_cop_ML,
                    'Mean_COP_displacement_AP[mm]':mean_cop_AP,
                    'std_COP_displacement_ML[mm]':std_cop_ML,
                    'std_COP_displacement_AP[mm]':std_cop_AP
                   }
        df_cop_stand_up.loc[len(df_cop_stand_up)] = new_line

In [ ]:
df_cop_stand_up

## COP Standing Upright

In [ ]:
df_cop_standing_upright = pd.DataFrame(columns=['Subject ID','Group',
                                                'Peak_COP_displacement_ML[mm]','Peak_COP_displacement_AP[mm]',
                                        'Mean_COP_displacement_ML[mm]','Mean_COP_displacement_AP[mm]',
                                        'std_COP_displacement_ML[mm]','std_COP_displacement_AP[mm]'
                                       ])

cop_data = [standing_control_cop,standing_pwp_cop]
data_label = ['Control','PwP']

for i,group in enumerate(cop_data):
    type_subject = data_label[i]
    for file in group:
                
       # Extract subject ID
        if type_subject == 'Control':
            subject_id = re.search(r'_C(\d+)\.txt$', file).group(1)
            subject_id = 'C'+subject_id
        else: 
            subject_id = re.search(r'_P(\d+)\.txt$', file).group(1)
            subject_id = 'P'+subject_id


        cop_file = file

        time, cop = read_cop_file(cop_file)
        cop -= cop[0]
        peak_cop_ML = max(np.abs(cop[:,0]))
        mean_cop_ML = np.mean(cop[:,0])
        std_cop_ML = np.std(cop[:,0])

        peak_cop_AP = max(np.abs(cop[:,1]))
        mean_cop_AP = np.mean(cop[:,1])
        std_cop_AP = np.std(cop[:,1])

        new_line = {'Subject ID':subject_id,'Group':type_subject,
                   'Peak_COP_displacement_ML[mm]':peak_cop_ML,
                    'Peak_COP_displacement_AP[mm]':peak_cop_AP,
                    'Mean_COP_displacement_ML[mm]':mean_cop_ML,
                    'Mean_COP_displacement_AP[mm]':mean_cop_AP,
                    'std_COP_displacement_ML[mm]':std_cop_ML,
                    'std_COP_displacement_AP[mm]':std_cop_AP
                   }
        df_cop_standing_upright.loc[len(df_cop_standing_upright)] = new_line

In [ ]:
df_cop_standing_upright

## COP displacement on Stand-to-sit task


In [ ]:

#Create the dataframes to store the shoulder tilt and stroop angles
df_cop_sit_down = pd.DataFrame(columns=['Subject ID','Group','TrialFMA',
                                   'Peak_COP_displacement_ML[mm]','Peak_COP_displacement_AP[mm]',
                                        'Mean_COP_displacement_ML[mm]','Mean_COP_displacement_AP[mm]',
                                        'std_COP_displacement_ML[mm]','std_COP_displacement_AP[mm]'
                                       ])

full_data = [fma_control_standtosit,fma_pwp_standtosit]
cop_data = [fma_control_cop,fma_pwp_cop]
data_label = ['Control','PwP']

for i,group in enumerate(cop_data):
    type_subject = data_label[i]
    for file in group:
                
       # Extract subject ID
        if type_subject == 'Control':
            subject_id = re.search(r'_C(\d+)\.txt$', file).group(1)
            subject_id = 'C'+subject_id
        else: 
            subject_id = re.search(r'_P(\d+)\.txt$', file).group(1)
            subject_id = 'P'+subject_id


        # Extract trial ID
        trial = re.search(r'FMA0(\d+)_', file).group(1)
        print(subject_id,trial)
        cop_file = file
        path = find_matching_file_path(cop_file,full_data[i])
#         print(cop_file)
#         print(path)
        peak_cop_ML, mean_cop_ML, std_cop_ML, peak_cop_AP, mean_cop_AP, std_cop_AP = get_AP_ML_cop(path,cop_file)

        new_line = {'Subject ID':subject_id,'Group':type_subject,'TrialFMA':trial,
                   'Peak_COP_displacement_ML[mm]':peak_cop_ML,
                    'Peak_COP_displacement_AP[mm]':peak_cop_AP,
                    'Mean_COP_displacement_ML[mm]':mean_cop_ML,
                    'Mean_COP_displacement_AP[mm]':mean_cop_AP,
                    'std_COP_displacement_ML[mm]':std_cop_ML,
                    'std_COP_displacement_AP[mm]':std_cop_AP
                   }
        df_cop_sit_down.loc[len(df_cop_sit_down)] = new_line
df_cop_sit_down